In [33]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import StratifiedKFold, GridSearchCV

In [34]:
customer_panel_df = pd.read_csv('/home/jovyan/shared/datasets/RetailBanking/customer_panel_train.csv')

In [35]:
# What are the proportions of the responses?
n = customer_panel_df.shape[0]
prop_default = (customer_panel_df['DefaultLabel'] == 1).sum() / n
print(f' Proportion of customers who default: {prop_default}')
print(f' Proportion of customers who do not default: {1-prop_default}')
# Dataset is extremely imbalanced - only 9.2% of customers in the data default

 Proportion of customers who default: 0.09232388542214871
 Proportion of customers who do not default: 0.9076761145778512


In [51]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Import train and test data
train_data = pd.read_csv('interactions_train.csv')
#train_data = pd.read_csv('customer_panel_train_merge.csv')

# For SMOTE:
smote = SMOTE(random_state=42, k_neighbors=5, sampling_strategy=0.5)
#train_data = train_data.fillna(-1)

features = ['PaymentRatio', 'TotalBalance', 'Limit',
             'Age', 'CreditScore', 'AnnualSalary', 'Utilisation', 'HardInquiries']
engineered_features = ['balance_to_limit',
                       'missed_payment_flag', 'total_accounts',
                       'credit_to_savings_ratio', 'balance_to_salary',
                       'limit_to_salary', 'payment_to_salary_ratio',
                        'SalaryBalance', 'SalaryCredit',
                      'CreditSquared', 'AgeCredit', 'AgeSalary', 'AgeBalance',
                       'PrevInquiries', 'LaggedUtilisation', 'LaggedPaymentRatio',
                       'RollingUtil', 'RollingPayRatio'] #
features = features + engineered_features

X_train_df = train_data[features]
y_train_df = train_data['DefaultLabel']
X, y = X_train_df.to_numpy(), y_train_df.to_numpy()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, stratify=y)

test_data = pd.read_csv('interactions_test.csv')
X_test_df = test_data[features]
X_test = X_test_df.to_numpy()

print(X_train.shape[0])
#X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

9310


In [53]:
from xgboost import XGBClassifier
## CV on XGB

# Calculate scale_pos_weight
neg_count = np.sum(y_train == 0)
pos_count = np.sum(y_train == 1)
scale_pos_weight = neg_count / pos_count # square root can be used if scale_pos_weight is too high
print(f'Scale weight value = {scale_pos_weight}')

# Define parameters for the xgboost model
params = {'objective':'binary:logistic',
          'eval_metric':'aucpr',
          'random_state':42,
          'scale_pos_weight': scale_pos_weight, # comment out if using smote
          'colsample_bylevel':1.0,
          'colsample_bynode':1.0
         }

# Define parameters to test using CV
param_grid = {'n_estimators':[700,800,900],
              'subsample':[0.6,0.7,0.8],
              'eta':[0.01,0.1],
              'max_depth':[5,6,7],
             }


# Instantiate model
xgb = XGBClassifier(**params)

# Perform CV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scorer = make_scorer(f1_score, average='macro')
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    cv=cv,
    scoring=scorer,
    n_jobs=-1,
    verbose=1
)
grid_search.fit(X_train, y_train)
#grid_search.fit(X_train_smote, y_train_smote)
xgb_best = grid_search.best_estimator_

y_hat_train = xgb_best.predict(X_train)
y_hat_val = xgb_best.predict(X_val)

F1_train = f1_score(y_train, y_hat_train, average='macro')
precision_train = precision_score(y_train, y_hat_train)
recall_train = recall_score(y_train, y_hat_train)

F1_val = f1_score(y_val, y_hat_val, average='macro')
precision_val = precision_score(y_val, y_hat_val)
recall_val = recall_score(y_val, y_hat_val)

print('----------------------------------------')
print(f'Best xgb F1-Score on Training Set: {F1_train}')
print(f'Best xgb Precision on Training Set: {precision_train}')
print(f'Best xgb Recall on Training Set: {recall_train}')
print('----------------------------------------')
print(f'Best xgb F1-Score on Validation Set: {F1_val}')
print(f'Best xgb Precision on Validation Set: {precision_val}')
print(f'Best xgb Recall on Validation Set: {recall_val}')
print('----------------------------------------')
print('Best xgb Parameters: ', grid_search.best_params_)

Scale weight value = 9.825581395348838
Fitting 5 folds for each of 54 candidates, totalling 270 fits
----------------------------------------
Best xgb F1-Score on Training Set: 0.9543133064414879
Best xgb Precision on Training Set: 0.8481262327416174
Best xgb Recall on Training Set: 1.0
----------------------------------------
Best xgb F1-Score on Validation Set: 0.94160948858746
Best xgb Precision on Validation Set: 0.848780487804878
Best xgb Recall on Validation Set: 0.9456521739130435
----------------------------------------
Best xgb Parameters:  {'eta': 0.01, 'max_depth': 7, 'n_estimators': 900, 'subsample': 0.7}


In [ ]:
#xgb_best.save_model('C2BestXGB2.json')

In [48]:
from sklearn.ensemble import RandomForestClassifier

## CV on RF

# Instantiate model
rf = RandomForestClassifier(random_state=42)

# Define parameters to test using CV
param_grid = {'max_depth': [8,9,10,11], 
             'min_samples_leaf': [1],
             'min_samples_split': [4],
             'max_features': [3,4,5,6],
             'n_estimators': [400,500,600]}  

# Perform CV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scorer = make_scorer(f1_score, average='macro')
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=cv,
    scoring=scorer,
    n_jobs=-1,
    verbose=1
)
grid_search.fit(X_train, y_train)
rf_best = grid_search.best_estimator_

y_hat_train = rf_best.predict(X_train)
y_hat_val = rf_best.predict(X_val)

F1_train = f1_score(y_train, y_hat_train, average='macro')
precision_train = precision_score(y_train, y_hat_train)
recall_train = recall_score(y_train, y_hat_train)

F1_val = f1_score(y_val, y_hat_val, average='macro')
precision_val = precision_score(y_val, y_hat_val)
recall_val = recall_score(y_val, y_hat_val)
print('----------------------------------------')
print(f'Best rf F1-Score on Training Set: {F1_train}')
print(f'Best rf Precision on Training Set: {precision_train}')
print(f'Best rf Recall on Training Set: {recall_train}')
print('----------------------------------------')
print(f'Best rf F1-Score on Validation Set: {F1_val}')
print(f'Best rf Precision on Validation Set: {precision_val}')
print(f'Best rf Recall on Validation Set: {recall_val}')
print('----------------------------------------')
print('Best rf Parameters: ', grid_search.best_params_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
----------------------------------------
Best rf F1-Score on Training Set: 0.9585967413687593
Best rf Precision on Training Set: 0.9611041405269761
Best rf Recall on Training Set: 0.8906976744186047
----------------------------------------
Best rf F1-Score on Validation Set: 0.9439172225136776
Best rf Precision on Validation Set: 0.9404761904761905
Best rf Recall on Validation Set: 0.8586956521739131
----------------------------------------
Best rf Parameters:  {'max_depth': 11, 'max_features': 6, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 400}


In [18]:
!pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [52]:
# Evaluate performance on train and validation sets using Macro F1-Score
y_hat_train = xgb.predict(X_train)
y_hat_val = xgb.predict(X_val)

F1_train = f1_score(y_train, y_hat_train, average='macro')
precision_train = precision_score(y_train, y_hat_train)
recall_train = recall_score(y_train, y_hat_train)

F1_val = f1_score(y_val, y_hat_val, average='macro')
precision_val = precision_score(y_val, y_hat_val)
recall_val = recall_score(y_val, y_hat_val)

print(f'F1-Score on Training Set: {F1_train}')
print(f'Precision on Training Set: {precision_train}')
print(f'Recall on Training Set: {recall_train}')
print('--------------------------------')
print(f'F1-Score on Validation Set: {F1_val}')
print(f'Precision on Validation Set: {precision_val}')
print(f'Recall on Validation Set: {recall_val}')

NotFittedError: need to call fit or load_model beforehand

In [49]:
from agentds import BenchmarkClient

# Generate submission file
client = BenchmarkClient(
    api_key="adsb_hdm1DRk1iW2I1VA84Oc9jz6z_1756090832",
    team_name="agi"
)

# Make predictions
predictions = rf_best.predict(X_test)

# Create submission file (format: CustomerID,Week,DefaultLabel)
submission_df = pd.DataFrame({
    'CustomerID': test_data['CustomerID'],
    'Week': test_data['Week'],
    'DefaultLabel': predictions
})

# Save predictions
submission_df.to_csv("retailbanking_challenge2_predictions.csv", index=False)
print(f"✅ Predictions saved: {submission_df.shape[0]} predictions")
print(f"   Preview: {submission_df.head(3)}")
print(f"   Default rate: {predictions.mean():.3f} ({predictions.sum()} defaults out of {len(predictions)})")

✅ Predictions saved: 13290 predictions
   Preview:   CustomerID  Week  DefaultLabel
0    C000001     1             0
1    C000001     2             0
2    C000001     3             0
   Default rate: 0.038 (500 defaults out of 13290)


In [50]:
# 3. Submit Predictions

# Submit predictions to the competition
print("🚀 Submitting predictions...")

try:
    result = client.submit_prediction("Retailbanking", 2, "retailbanking_challenge2_predictions.csv")
    
    if result['success']:
        print("✅ Submission successful!")
        print(f"   📊 Score: {result['score']:.4f}")
        print(f"   📏 Metric: {result['metric_name']}")
        print(f"   ✔️  Validation: {'Passed' if result['validation_passed'] else 'Failed'}")
    else:
        print("❌ Submission failed!")
        print(f"   Error details: {result.get('details', {}).get('validation_errors', 'Unknown error')}")
        
except Exception as e:
    print(f"💥 Submission error: {e}")
    print("🔧 Check your API key and team name are correct!")

print("\n🎯 Next steps:")
print("   1. Try incorporating relevant information outside this table!")
print("   2. You've completed all Retail Banking challenges!")


🚀 Submitting predictions...
✅ Prediction submitted successfully!
📊 Score: 0.6595 (Macro-F1)
✅ Validation passed
✅ Submission successful!
   📊 Score: 0.6595
   📏 Metric: Macro-F1
   ✔️  Validation: Passed

🎯 Next steps:
   1. Try incorporating relevant information outside this table!
   2. You've completed all Retail Banking challenges!


In [ ]:
# Which of the features are highly correlated with each other?

In [48]:
from scipy.stats import ttest_ind
from scipy.stats import chi2_contingency

# Which of the features are highly associated with the response?

categoricals = ['is_high_inquiry_user', 'is_rapid_credit_growth']
# For quantitative predictors use T-test to see if mean values differ across the response
for feature in list(set(features)-set(categoricals)):
    group1 = train_data[train_data['DefaultLabel']==1].dropna()
    group2 = train_data[train_data['DefaultLabel']==0].dropna()
    ttest = ttest_ind(group1[feature], group2[feature])
    if(ttest.pvalue > 0.05):
        print(feature)
        print(f'T-Stat: {ttest.statistic}')
        print(f'p-val: {ttest.pvalue}')
        print('--------------')
# Can drop Utilisation, HardInquiries, NumChecking, Tenure, payment_ratio_change, credit_utilization_change, utilization_trend, missed_payment_flag, HomeCity

# Chi^2
# Create the contingency table
# contingency_table = pd.crosstab(train_data['is_high_inquiry_user'], train_data['DefaultLabel']) p = 3e-25
# contingency_table = pd.crosstab(train_data['is_rapid_credit_growth'], train_data['DefaultLabel']) p = 0.45
#contingency_table = pd.crosstab(train_data['is_high_utilization'], train_data['DefaultLabel'])# p = 1.0
contingency_table = pd.crosstab(train_data['missed_payment_flag'], train_data['DefaultLabel'])# p = 0.45
print("Contingency Table:")
print(contingency_table)
chi2_statistic, p_value, degrees_of_freedom, expected_frequencies = chi2_contingency(contingency_table)
print(f"\nChi-squared Statistic: {chi2_statistic}")
print(f"P-value: {p_value}")
print(f"Degrees of Freedom: {degrees_of_freedom}")
print("Expected Frequencies:")
print(expected_frequencies)
# Can drop is_rapid_credit_growth

Utilisation
T-Stat: 0.9781635715541426
p-val: 0.3280357898449866
--------------
missed_payment_flag
T-Stat: 0.21306032130528654
p-val: 0.8312876879453321
--------------
Contingency Table:
DefaultLabel            0    1
missed_payment_flag           
0                    5271  302
1                    6802  926

Chi-squared Statistic: 165.66889238284364
P-value: 6.534218501583134e-38
Degrees of Freedom: 1
Expected Frequencies:
[[5058.47898654  514.52101346]
 [7014.52101346  713.47898654]]
